In [ ]:
%%writefile ../../.devcontainer/.env.template
# =============================================================================
# CONTAINER CONFIGURATION - Primary Settings
# =============================================================================
ENV_NAME=docker_dev_template

# =============================================================================
# DOCKER & CUDA CONFIGURATION  
# =============================================================================
CUDA_TAG=12.8.0
DOCKER_BUILDKIT=1

# =============================================================================
# HOST PORT MAPPINGS
# =============================================================================
HOST_JUPYTER_PORT=8895
HOST_TENSORBOARD_PORT=6005
HOST_EXPLAINER_PORT=8055
HOST_STREAMLIT_PORT=8505
HOST_MLFLOW_PORT=5005

# =============================================================================
# PYTHON & RUNTIME CONFIGURATION
# =============================================================================
PYTHON_VER=3.10

# =============================================================================
# GPU MEMORY MANAGEMENT (PyTorch + JAX Only)
# =============================================================================
JAX_PLATFORM_NAME=

# Optimized for RTX 4090 - PyTorch + JAX memory sharing
XLA_PYTHON_CLIENT_PREALLOCATE=false
XLA_PYTHON_CLIENT_ALLOCATOR=platform  
XLA_PYTHON_CLIENT_MEM_FRACTION=0.35
XLA_FLAGS=--xla_force_host_platform_device_count=1
JAX_DISABLE_JIT=false
JAX_ENABLE_X64=false
JAX_PREALLOCATION_SIZE_LIMIT_BYTES=10737418240

# PyTorch memory management for RTX 4090
PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024,expandable_segments:True,roundup_power2_divisions:16

# =============================================================================
# JUPYTER CONFIGURATION
# =============================================================================
JUPYTER_TOKEN=jupyter

# =============================================================================
# UV PACKAGE MANAGER CONFIGURATION
# =============================================================================
UV_PROJECT_ENVIRONMENT=/app/.venv

# =============================================================================
# COMPUTER VISION CONFIGURATION - SIMPLIFIED
# =============================================================================
# Roboflow API Key - Get from https://app.roboflow.com/settings/api
# IMPORTANT: Replace the placeholder below with your actual API key
ROBOFLOW_API_KEY=your_roboflow_api_key_here

# Computer Vision Environment Variables
YOLO_VERBOSE=false
OPENCV_LOG_LEVEL=ERROR

# Display Configuration for GUI Support
DISPLAY=:0
QT_X11_NO_MITSHM=1
LIBGL_ALWAYS_INDIRECT=1

# Video Processing Directories
VIDEO_INPUT_DIR=/workspace/videos/input
VIDEO_OUTPUT_DIR=/workspace/videos/output

# =============================================================================
# KAGGLE AUTHENTICATION (Optional)
# =============================================================================
KAGGLE_USERNAME=geoffhadfield 
KAGGLE_KEY=your_api_token


Overwriting ../../.devcontainer/.env.template


In [2]:
%%writefile ../../.devcontainer/.dockerignore
# Reduce Docker build context
.git
.gitignore
.gitattributes
.gitmodules
.vscode
.idea
*.swp
*.swo
*~
.DS_Store
Thumbs.db
__pycache__
*.pyc
*.pyo
*.pyd
.Python
*.so
.coverage*
.cache
.pytest_cache
.mypy_cache
.tox
pip-log.txt
pip-delete-this-directory.txt
env
venv
ENV
env.bak
venv.bak
.ipynb_checkpoints
# Large data (adjust as needed)
data/raw
data/external
*.csv
*.parquet
*.h5
*.hdf5
# Models
*.pt
*.pth
*.pkl
*.joblib
models/
# Logs and temps
*.log
logs/
*.tmp
*.temp
.tmp
temp/
# Build artifacts
build/
dist/
*.egg-info/
.eggs/
# Node
node_modules
npm-debug.log*
yarn-*.log*
.npm
.eslintcache
.node_repl_history
*.tgz
*.tar.gz
# Archives
*.zip
*.tar
*.tar.bz2
*.rar
*.7z
# Docs (opt‑in if needed)
docs/
*.md
README*
LICENSE*
CHANGELOG*
# Tests (opt‑in if needed)
tests/
test_*
*_test.py
# CI
.github/
.gitlab-ci.yml
.travis.yml
.circleci/
azure-pipelines.yml
# Env
.env
.env.local
.env.*.local
.editorconfig
.prettierrc*
.eslintrc*
# Universal junk (de‑duped)
*.py[cod]

Overwriting ../../.devcontainer/.dockerignore


In [3]:
%%writefile ../../.devcontainer/devcontainer.json
{
  "name": "docker_dev_template_rtx4090",
  "dockerComposeFile": "../docker-compose.yml",
  "service": "datascience",
  "workspaceFolder": "/workspace",
  "shutdownAction": "stopCompose",

  "overrideCommand": false,
  "containerEnv": {
    "CONTAINER_WORKSPACE_FOLDER": "/workspace",
    "UV_PROJECT_ENVIRONMENT": "/app/.venv",
    "VIRTUAL_ENV": "/app/.venv",
    "PYTHONPATH": "/workspace",
    "TERM": "xterm-256color"
  },

  "runArgs": [
    "--gpus", "all",
    "--name", "${localEnv:ENV_NAME:docker_dev_template}_datascience"
  ],

  "customizations": {
    "vscode": {
      "settings": {
        "python.defaultInterpreterPath": "/app/.venv/bin/python",
        "python.pythonPath": "/app/.venv/bin/python",
        "python.terminal.activateEnvironment": true,
        "python.terminal.activateEnvInCurrentTerminal": true,
        "terminal.integrated.defaultProfile.linux": "bash",
        "terminal.integrated.profiles.linux": {
          "bash": {
            "path": "/bin/bash",
            "args": ["-l"],
            "env": {
              "VIRTUAL_ENV": "/app/.venv",
              "PATH": "/app/.venv/bin:${env:PATH}",
              "UV_PROJECT_ENVIRONMENT": "/app/.venv",
              "PYTHONPATH": "/workspace"
            }
          }
        },
        "jupyter.notebookFileRoot": "/workspace",
        "jupyter.kernels.filter": [
          {
            "path": "/app/.venv/bin/python",
            "type": "pythonEnvironment"
          }
        ]
      },
      "extensions": [
        "ms-python.python",
        "ms-toolsai.jupyter",
        "ms-azuretools.vscode-docker",
        "ms-python.flake8",
        "ms-python.black-formatter"
      ]
    }
  },

  "onCreateCommand": [
    "bash", "-lc",
    "echo 'onCreate: validating environment'; ls -la /app/.venv/bin/; which python || echo 'python not found in PATH'"
  ],

  "postCreateCommand": [
    "bash", "-lc",
    "set -e; source /app/.venv/bin/activate; python -c 'import sys; print(f\"python: {sys.executable}\")'; uv pip install -U ipykernel jupyter-client -q; python -m ipykernel install --user --name='uv_docker_dev_template' --display-name='Python (UV Environment)'; jupyter kernelspec list; python /app/tests/test_summary.py"
  ],

  "postStartCommand": [
    "bash", "-lc",
    "source /app/.venv/bin/activate; python --version; python -c 'import torch; print(f\"pytorch cuda: {torch.cuda.is_available()}\")' || echo 'pytorch test failed'; python /app/validate_gpu.py --quick || echo 'gpu validation completed with warnings'"
  ],

  "features": {},
  "forwardPorts": [8888, 6008, 8050, 8501, 5000],
  "portsAttributes": {
    "8888": { "label": "Jupyter Lab", "onAutoForward": "notify" },
    "6008": { "label": "TensorBoard", "onAutoForward": "silent" },
    "8050": { "label": "Explainer Dashboard", "onAutoForward": "silent" },
    "8501": { "label": "Streamlit", "onAutoForward": "silent" },
    "5000": { "label": "MLflow", "onAutoForward": "silent" }
  },

  "mounts": [
    "source=docker_dev_template_uv_cache,target=/root/.cache/uv,type=volume"
  ]
}

Overwriting ../../.devcontainer/devcontainer.json


In [4]:
%%writefile ../../.devcontainer/Dockerfile
# Dockerfile: RTX 4090 devcontainer with UV, JAX, and PyTorch (CUDA 12.x)

ARG CUDA_TAG=12.4.0
FROM nvidia/cuda:${CUDA_TAG}-devel-ubuntu22.04

ARG PYTHON_VER=3.10
ARG ENV_NAME=docker_dev_template
ENV DEBIAN_FRONTEND=noninteractive

# System dependencies with Computer Vision additions
RUN --mount=type=cache,id=apt-cache,target=/var/cache/apt,sharing=locked \
    --mount=type=cache,id=apt-lists,target=/var/lib/apt/lists,sharing=locked \
    apt-get update && apt-get install -y --no-install-recommends \
        bash curl ca-certificates git procps htop \
        python3 python3-venv python3-pip python3-dev \
        build-essential cmake pkg-config \
        libjemalloc2 libjemalloc-dev \
        iproute2 net-tools lsof wget \
        # Computer Vision dependencies
        ffmpeg \
        libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1 \
        libgstreamer1.0-0 libgstreamer-plugins-base1.0-0 \
        libgtk-3-0 libgtk-3-dev \
        # X11 support for GUI applications
        x11-apps xauth xvfb \
        # Video codec libraries
        libavcodec-dev libavformat-dev libswscale-dev \
        libv4l-dev libxvidcore-dev libx264-dev \
        # Image format libraries
        libjpeg-dev libpng-dev libtiff-dev \
        # OpenGL support for visualization
        libgl1-mesa-glx libglu1-mesa-dev \
    && apt-get clean && rm -rf /var/lib/apt/lists/*

# UV package manager
COPY --from=ghcr.io/astral-sh/uv:0.7.12 /uv /uvx /bin/

WORKDIR /app

# Create venv managed by UV
RUN uv venv .venv --python "${PYTHON_VER}" --prompt "${ENV_NAME}"

ENV VIRTUAL_ENV=/app/.venv \
    PATH="/app/.venv/bin:${PATH}" \
    UV_PROJECT_ENVIRONMENT=/app/.venv \
    PYTHONPATH="/workspace"

# Memory and allocator settings
ENV LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2 \
    MALLOC_ARENA_MAX=2 \
    MALLOC_TCACHE_MAX=0 \
    PYTORCH_NO_CUDA_MEMORY_CACHING=1

# GPU‑relevant environment
ENV XLA_PYTHON_CLIENT_PREALLOCATE=false \
    XLA_PYTHON_CLIENT_MEM_FRACTION=0.4 \
    XLA_PYTHON_CLIENT_ALLOCATOR=platform \
    PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024,expandable_segments:True \
    JAX_PREALLOCATION_SIZE_LIMIT_BYTES=17179869184

# Computer Vision specific environment variables
ENV OPENCV_VIDEOIO_PRIORITY_GSTREAMER=0 \
    QT_X11_NO_MITSHM=1 \
    DISPLAY=:0

# Create directories for models and data
RUN mkdir -p /app/models /app/data /app/weights \
    && chmod 755 /app/models /app/data /app/weights

# Bring in project descriptors and tests
COPY pyproject.toml /workspace/
COPY uv.lock* /workspace/
COPY .devcontainer/validate_gpu.py /app/validate_gpu.py
COPY .devcontainer/tests/ /app/tests/

# Resolve project dependencies with UV
RUN --mount=type=cache,target=/root/.cache/uv,sharing=locked \
    cd /workspace && (uv sync --frozen --no-dev || (uv sync --no-dev && uv lock))

# CRITICAL FIX 2: Install PyTorch first to establish CUDA environment
RUN --mount=type=cache,target=/root/.cache/uv,sharing=locked \
    echo "Installing PyTorch with CUDA 12.4..." && \
    uv pip install --no-cache-dir torch torchvision torchaudio \
        --index-url https://download.pytorch.org/whl/cu124

# CRITICAL FIX 3: Install compatible CuDNN 9.8.0 to satisfy JAX requirements
RUN --mount=type=cache,target=/root/.cache/uv,sharing=locked \
    echo "Upgrading CuDNN to 9.8.0 for JAX compatibility..." && \
    uv pip install --no-cache-dir --upgrade nvidia-cudnn-cu12==9.8.0.69 || \
    uv pip install --no-cache-dir --upgrade nvidia-cudnn-cu12>=9.8.0

# CRITICAL FIX 4: Install JAX after CuDNN upgrade with proper dependency resolution
RUN --mount=type=cache,target=/root/.cache/uv,sharing=locked \
    echo "Removing any existing JAX installations..." && \
    (uv pip uninstall jax jaxlib jax-cuda12-plugin jax-cuda12-pjrt || true) && \
    echo "Installing JAX with CUDA 12 support..." && \
    (uv pip install --no-cache-dir "jax[cuda12-local]>=0.4.26" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html \
     || uv pip install --no-cache-dir "jax[cpu]>=0.4.26")

# Install Computer Vision specific packages
RUN --mount=type=cache,target=/root/.cache/uv,sharing=locked \
    echo "Installing Computer Vision packages..." && \
    # Ensure latest Ultralytics and dependencies
    uv pip install --no-cache-dir --upgrade ultralytics==8.3.158 && \
    # Install tracking libraries
    uv pip install --no-cache-dir supervision>=0.17.0 lap>=0.4.0 && \
    # Install additional CV utilities
    uv pip install --no-cache-dir albumentations>=1.3.0 && \
    # Ensure opencv-contrib for advanced features
    uv pip install --no-cache-dir opencv-contrib-python-headless>=4.10.0

# REMOVED: Pre-download YOLO models (models will be downloaded on first use)
# This reduces build time and container size significantly
# Models will be cached in the mounted volumes during runtime

# Jupyter kernel support
RUN --mount=type=cache,target=/root/.cache/uv,sharing=locked \
    uv pip install ipykernel jupyter-client jupyterlab

# CUDA libs in path - include both system and package CUDA libraries
ENV LD_LIBRARY_PATH="/app/.venv/lib:/app/.venv/lib/python3.10/site-packages/nvidia/cudnn/lib:/usr/local/cuda/lib64:${LD_LIBRARY_PATH}"

# Shell activation helper with updated library paths
RUN echo '#!/bin/bash' > /app/activate_uv.sh && \
    echo 'export VIRTUAL_ENV="/app/.venv"' >> /app/activate_uv.sh && \
    echo 'export PATH="/app/.venv/bin:$PATH"' >> /app/activate_uv.sh && \
    echo 'export UV_PROJECT_ENVIRONMENT="/app/.venv"' >> /app/activate_uv.sh && \
    echo 'export PYTHONPATH="/workspace:$PYTHONPATH"' >> /app/activate_uv.sh && \
    echo 'export LD_LIBRARY_PATH="/app/.venv/lib:/app/.venv/lib/python3.10/site-packages/nvidia/cudnn/lib:/usr/local/cuda/lib64:${LD_LIBRARY_PATH}"' >> /app/activate_uv.sh && \
    echo '# Computer Vision environment' >> /app/activate_uv.sh && \
    echo 'export YOLO_VERBOSE=false' >> /app/activate_uv.sh && \
    echo 'export OPENCV_LOG_LEVEL=ERROR' >> /app/activate_uv.sh && \
    echo 'cd /workspace' >> /app/activate_uv.sh && \
    chmod +x /app/activate_uv.sh && \
    echo 'source /app/activate_uv.sh' > /etc/profile.d/10-uv-activate.sh && \
    echo 'source /app/activate_uv.sh' >> /root/.bashrc && \
    chmod +x /etc/profile.d/10-uv-activate.sh

# Enhanced healthcheck with CV components (updated without YOLO model check)
RUN echo '#!/bin/bash' > /app/healthcheck.sh && \
    echo 'source /app/.venv/bin/activate' >> /app/healthcheck.sh && \
    echo 'echo "=== Environment Check ==="' >> /app/healthcheck.sh && \
    echo 'python --version' >> /app/healthcheck.sh && \
    echo 'echo "=== CuDNN Version Check ==="' >> /app/healthcheck.sh && \
    echo 'python -c "import torch; print(f\"PyTorch CuDNN: {torch.backends.cudnn.version()}\")" || echo "PyTorch CuDNN check failed"' >> /app/healthcheck.sh && \
    echo 'echo "=== JAX Device Check ==="' >> /app/healthcheck.sh && \
    echo 'python -c "import jax; print(f\"JAX devices: {jax.devices()}\")" || echo "JAX device check failed"' >> /app/healthcheck.sh && \
    echo 'echo "=== Computer Vision Check ==="' >> /app/healthcheck.sh && \
    echo 'python -c "import cv2; print(f\"OpenCV: {cv2.__version__}\")" || echo "OpenCV check failed"' >> /app/healthcheck.sh && \
    echo 'python -c "from ultralytics import YOLO; print(\"YOLO package: OK\")" || echo "YOLO package check failed"' >> /app/healthcheck.sh && \
    echo 'python -c "import roboflow; print(f\"Roboflow: {roboflow.__version__}\")" || echo "Roboflow check failed"' >> /app/healthcheck.sh && \
    echo 'echo "=== GPU Validation ==="' >> /app/healthcheck.sh && \
    echo 'python /app/validate_gpu.py --quick' >> /app/healthcheck.sh && \
    chmod +x /app/healthcheck.sh

WORKDIR /workspace
CMD ["bash", "-l"]

Overwriting ../../.devcontainer/Dockerfile


In [5]:
%%writefile ../../docker-compose.yml
name: ${ENV_NAME:-docker_dev_template}

services:
  datascience:
    build:
      context: .
      dockerfile: .devcontainer/Dockerfile
      args:
        CUDA_TAG: ${CUDA_TAG:-12.4.0}
        PYTHON_VER: ${PYTHON_VER:-3.10}
        ENV_NAME: ${ENV_NAME:-docker_dev_template}
      cache_from:
        - nvidia/cuda:${CUDA_TAG:-12.4.0}-devel-ubuntu22.04

    container_name: ${ENV_NAME:-docker_dev_template}_datascience

    # UPDATED: Reference environment template from .devcontainer folder
    env_file:
      - .devcontainer/.env

    restart: unless-stopped
    depends_on:
      mlflow:
        condition: service_healthy

    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: all
              capabilities: [gpu, compute, utility, video]  # Added video capability

    init: true
    gpus: all
    shm_size: 16g  # Increased for video processing
    ulimits:
      memlock: -1
      stack: 67108864

    environment:
      - PYTHON_VER=${PYTHON_VER:-3.10}
      - UV_PROJECT_ENVIRONMENT=/app/.venv
      - VIRTUAL_ENV=/app/.venv
      - PYTHONPATH=/workspace
      - NVIDIA_VISIBLE_DEVICES=all
      - NVIDIA_DRIVER_CAPABILITIES=compute,utility,video
      - CUDA_VISIBLE_DEVICES=0
      - LD_LIBRARY_PATH=/app/.venv/lib:/usr/local/cuda/lib64
      - LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2
      - MALLOC_ARENA_MAX=2
      - MALLOC_TCACHE_MAX=0
      - PYTORCH_NO_CUDA_MEMORY_CACHING=1
      
      # CRITICAL FIX: Removed inline comments from JAX environment variables
      # These were causing "could not convert string to float" errors
      - XLA_PYTHON_CLIENT_PREALLOCATE=false
      - XLA_PYTHON_CLIENT_ALLOCATOR=platform
      - XLA_PYTHON_CLIENT_MEM_FRACTION=0.4
      - XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda
      - JAX_PREALLOCATION_SIZE_LIMIT_BYTES=17179869184
      - PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024,expandable_segments:True
      - JUPYTER_TOKEN=${JUPYTER_TOKEN:-jupyter}
      
      # Computer Vision configuration
      - YOLO_VERBOSE=${YOLO_VERBOSE:-false}
      - OPENCV_LOG_LEVEL=${OPENCV_LOG_LEVEL:-ERROR}
      - ROBOFLOW_API_KEY=${ROBOFLOW_API_KEY:-}
      - ULTRALYTICS_HUB_API_KEY=${ULTRALYTICS_HUB_API_KEY:-}
      
      # Display configuration for GUI support
      - DISPLAY=${DISPLAY:-:0}
      - QT_X11_NO_MITSHM=1
      - LIBGL_ALWAYS_INDIRECT=1

    volumes:
      # Main workspace
      - .:/workspace:delegated
      
      # MLflow artifacts
      - ./mlruns:/workspace/mlruns
      
      # UV cache
      - uv-cache:/root/.cache/uv
      
      # Computer Vision specific volumes
      - ./models:/app/models:delegated
      - ./weights:/app/weights:delegated
      - ./data:/app/data:delegated
      - ./videos:/workspace/videos:delegated
      
      # Ultralytics/YOLO cache
      - yolo-cache:/root/.cache/ultralytics
      
      # Roboflow cache
      - roboflow-cache:/root/.cache/roboflow
      
      # X11 socket for GUI (Linux/Unix only)
      - /tmp/.X11-unix:/tmp/.X11-unix:rw
      - ${HOME}/.Xauthority:/root/.Xauthority:rw

    ports:
      # Jupyter Lab
      - "${HOST_JUPYTER_PORT:-8891}:8888"
      
      # TensorBoard
      - "${HOST_TENSORBOARD_PORT:-6008}:6008"
      
      # Explainer Dashboard
      - "${HOST_EXPLAINER_PORT:-8050}:8050"
      
      # Streamlit
      - "${HOST_STREAMLIT_PORT:-8501}:8501"
      
      # Computer Vision specific ports
      - "${HOST_CV_API_PORT:-8080}:8080"  # CV API server
      - "${HOST_CV_STREAM_PORT:-8554}:8554"  # RTSP stream

    command: >
      bash -lc '
        echo "[boot] Starting container: ${ENV_NAME:-docker_dev_template}";
        echo "[boot] Activating uv environment...";
        source /app/.venv/bin/activate;
        echo "[boot] Environment activated - Python: $(which python)";
        echo "[boot] UV available: $(uv --version)";
        echo "[boot] Running GPU validation...";
        python /app/validate_gpu.py || echo "GPU validation warning - check logs";
        echo "[boot] Checking Computer Vision components...";
        python /app/tests/test_cv.py || echo "CV check warning - check logs";
        echo "[boot] Starting Jupyter Lab on port 8888...";
        jupyter lab --ip=0.0.0.0 --port=8888 --allow-root 
        --NotebookApp.token="${JUPYTER_TOKEN}" 
        --NotebookApp.allow_origin="*" 
        --NotebookApp.open_browser=false
      '

    healthcheck:
      test:
        - CMD-SHELL
        - |
          python -c '
          import torch, jax, cv2
          from ultralytics import YOLO
          assert torch.cuda.is_available()
          assert any("gpu" in str(d).lower() for d in jax.devices())
          assert cv2.__version__ is not None
          print("All systems operational")
          ' 2>/dev/null || exit 1
      interval: 60s
      timeout: 30s
      retries: 3
      start_period: 120s

    labels:
      - "com.docker.compose.project=${ENV_NAME:-docker_dev_template}"
      - "com.docker.compose.service=datascience"
      - "description=RTX 4090 GPU Dev Environment with Computer Vision (PyTorch+JAX+YOLO) - CUDA 12.4"

  mlflow:
    container_name: ${ENV_NAME:-docker_dev_template}_mlflow
    image: ghcr.io/mlflow/mlflow:latest
    command: >
      mlflow server
      --host 0.0.0.0
      --port 5000
      --backend-store-uri sqlite:///mlflow.db
      --default-artifact-root /mlflow_artifacts
    environment:
      MLFLOW_EXPERIMENTS_DEFAULT_ARTIFACT_LOCATION: /mlflow_artifacts
    volumes:
      - ./mlruns:/mlflow_artifacts
      - ./mlflow_db:/mlflow_db
    ports:
      - "${HOST_MLFLOW_PORT:-5000}:5000"
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "python", "-c", "import requests; requests.get('http://localhost:5000/health').raise_for_status()"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 30s

volumes:
  uv-cache:
    driver: local
  yolo-cache:
    driver: local
  roboflow-cache:
    driver: local


Overwriting ../../docker-compose.yml


In [6]:
%%writefile ../../pyproject.toml
[project]
name = "docker_dev_template"
version = "0.1.0"
description = "Hierarchical Bayesian modeling for baseball exit velocity data"
authors = [
  { name = "Marlins Data Science Team" },
]
license = "MIT"
readme = "README.md"

# ─── Restrict to Python 3.10–3.12 ──────────────────────────────
requires-python = ">=3.10,<3.13"

dependencies = [
  "pandas>=2.0",
  "numpy>=1.20,<2",
  "matplotlib>=3.4.0",
  "scikit-learn>=1.4.2",
  "pymc>=5.0.0",
  "arviz>=0.14.0",
  "statsmodels>=0.13.0",
  "jupyterlab>=3.0.0",
  "seaborn>=0.11.0",
  "tabulate>=0.9.0",
  "shap>=0.40.0",
  "xgboost>=1.5.0",
  "lightgbm>=3.3.0",
  "catboost>=1.0.0",
  "scipy>=1.7.0",
  "shapash[report]>=2.3.0",
  "shapiq>=1.3.0",
  "explainerdashboard>=0.3.0",
  "ipywidgets>=8.0.0",
  "nutpie>=0.7.1",
  "numpyro>=0.18.0,<1.0.0",
  "jax>=0.4.23",
  "jaxlib>=0.4.23",
  "pytensor>=2.18.3",
  "aesara>=2.9.4",
  "tqdm>=4.67.0",
  "pyarrow>=12.0.0",
  "streamlit>=1.20.0",
  "sqlalchemy>=1.4",
  "mysql-connector-python>=8.0",
  "optuna>=4.3.0",
  "bayesian-optimization>=1.2.0",
  "pretty_errors>=1.2.0",
  "gdown>=4.0.0",
  "invoke>=2.2",
  # ▶ Video download stack
  #   - pytube main-branch until next PyPI release (optional fallback)
  "pytube @ git+https://github.com/pytube/pytube",
  "yt-dlp==2025.9.5",  # Pinned for stability
  #   - optional convenience wrapper (does NOT install ffmpeg binary!)
  "ffmpeg-python==0.2.0",  # Pinned for stability

  # ▶ Computer Vision Stack
  # Core CV libraries
  "ultralytics==8.3.158",  # YOLO v8 - pinned for stability
  "opencv-contrib-python-headless>=4.10.0",  # OpenCV with contrib modules
  "roboflow==1.2.9",  # Roboflow API client - pinned for stability
  
  # Object tracking and supervision
  "supervision>=0.17.0",  # Modern CV utilities and tracking
  "lap>=0.4.0",  # Linear Assignment Problem solver for tracking
  
  # Image augmentation and processing
  "albumentations>=1.3.0",  # Fast image augmentation
  "imgaug>=0.4.0",  # Image augmentation library
  "pillow>=10.0.0",  # Image processing
  
  # Video processing and download
  "moviepy==2.2.1",  # Video editing - pinned for stability
  "mlflow>=3.1.1,<4.0.0",
  "optuna-integration[mlflow]>=4.4.0,<5.0.0",
  
  # PyTorch core libraries - platform specific with PEP-508 compliant syntax
  # CUDA wheels for Windows/Linux, CPU for macOS
"torch>=2.0.0",
"torchvision>=0.15.0",
"torchaudio>=2.0.0",
]

[project.optional-dependencies]
dev = [
  "pytest>=7.0.0",
  "black>=23.0.0",
  "isort>=5.0.0",
  "flake8>=5.0.0",
  "mypy>=1.0.0",
  "pre-commit>=3.0.0",
]

cuda = [
  "cupy-cuda12x>=12.0.0",  # For CUDA 12.x
]

basketball = [
  # Additional basketball-specific packages
  "sportsipy>=0.6.0",  # Sports statistics
  "nba-api>=1.4.0",  # NBA API client
]

# ─── uv configuration ──────────────────────────────────────────
[tool.uv]                   # uv reads this block
index-strategy = "unsafe-best-match"

# Define named indexes for PyTorch CUDA variants
[[tool.uv.index]]
name = "pytorch-cu121"
url = "https://download.pytorch.org/whl/cu121"
explicit = true

[[tool.uv.index]]
name = "pytorch-cu118"
url = "https://download.pytorch.org/whl/cu118"
explicit = true

[[tool.uv.index]]
name = "pytorch-cu124"
url = "https://download.pytorch.org/whl/cu124"
explicit = true

[[tool.uv.index]]
name = "pytorch-cu128"
url = "https://download.pytorch.org/whl/cu128"
explicit = true

# Removed unsupported option: torch-backend requires uv ≥0.5.3
# To re-enable, first run: pip install -U uv>=0.5.3
[tool.uv.pip]
# (No unsupported keys here; configure only valid pip options.)

# Map PyTorch dependencies to CUDA indexes for non-macOS platforms
# Testing with CUDA 12.8
[tool.uv.sources]
torch = [
  { index = "pytorch-cu128", marker = "sys_platform == 'linux' or sys_platform == 'win32'" },
]
torchvision = [
  { index = "pytorch-cu128", marker = "sys_platform == 'linux' or sys_platform == 'win32'" },
]
torchaudio = [
  { index = "pytorch-cu128", marker = "sys_platform == 'linux' or sys_platform == 'win32'" },
]

[tool.pytensor]
device    = "cuda"
floatX    = "float32"
allow_gc  = true
optimizer = "fast_run"


Overwriting ../../pyproject.toml


In [7]:
%%writefile ../../.devcontainer/validate_gpu.py
#!/usr/bin/env python3
"""
GPU validation and environment diagnostics for RTX 4090 devcontainer.
Focus: verify JAX and PyTorch access to CUDA, report common misconfigurations.
"""
import sys
import os
import subprocess
import warnings
import textwrap
import re
warnings.filterwarnings('ignore')


def print_section(title: str) -> None:
    print("\n" + "=" * 60)
    print(f"  {title}")
    print("=" * 60)


def validate_environment_variables() -> bool:
    """Validate JAX‑related environment variables (no inline comments, valid types)."""
    print_section("JAX ENVIRONMENT VARIABLE VALIDATION")

    jax_numeric_vars = {
        'XLA_PYTHON_CLIENT_MEM_FRACTION': {'type': 'float', 'range': (0.0, 1.0)},
        'JAX_PREALLOCATION_SIZE_LIMIT_BYTES': {'type': 'int', 'range': (0, None)},
    }
    jax_string_vars = {
        'XLA_FLAGS', 'JAX_PLATFORM_NAME', 'XLA_PYTHON_CLIENT_ALLOCATOR', 'XLA_PYTHON_CLIENT_PREALLOCATE'
    }

    ok = True
    problems = []

    for var, cfg in jax_numeric_vars.items():
        value = os.environ.get(var)
        print(f"\nCheck {var} -> {value}")
        if value is None:
            print("  not set; defaults apply")
            continue
        if '#' in value:
            clean = value.split('#')[0].strip()
            print("  contains inline comment; use:", clean)
            problems.append((var, value, clean))
            ok = False
            continue
        try:
            if cfg['type'] == 'float':
                v = float(value)
                low, high = cfg['range']
                if (low is not None and v < low) or (high is not None and v > high):
                    print("  out of recommended range")
                else:
                    print("  ok")
            else:
                v = int(value)
                print("  ok")
        except ValueError as e:
            print("  invalid numeric value:", e)
            ok = False

    for var in jax_string_vars:
        value = os.environ.get(var)
        if value and '#' in value:
            print(f"warn: {var} contains '#', which can break parsing")

    if problems:
        print("\nFix suggestions:")
        for var, bad, clean in problems:
            print(f"export {var}={clean}")
    return ok


def check_environment() -> None:
    print_section("ENVIRONMENT CHECK")
    print("python:", sys.executable)
    print("version:", sys.version)
    print("VIRTUAL_ENV:", os.environ.get('VIRTUAL_ENV'))
    print("PATH contains .venv:", '.venv/bin' in os.environ.get('PATH', ''))

    cuda_vars = ['CUDA_HOME', 'CUDA_PATH', 'CUDA_VISIBLE_DEVICES', 'LD_LIBRARY_PATH', 'NVIDIA_VISIBLE_DEVICES']
    print("\nCUDA variables:")
    for var in cuda_vars:
        print(f"  {var}:", os.environ.get(var, 'not set'))

    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=name,driver_version,memory.total', '--format=csv,noheader'],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("\nGPU:", result.stdout.strip())
        else:
            print("\nwarn: nvidia-smi returned non‑zero")
    except FileNotFoundError:
        print("\nwarn: nvidia-smi not found in path")


def test_pytorch() -> bool:
    print_section("PYTORCH GPU TEST")
    try:
        import torch
        print("version:", torch.__version__)
        print("cuda available:", torch.cuda.is_available())
        if torch.cuda.is_available():
            print("device count:", torch.cuda.device_count())
            print("device 0:", torch.cuda.get_device_name(0))
            # quick matmul
            import time
            dev = torch.device('cuda')
            x = torch.randn(2000, 2000, device=dev)
            y = torch.randn(2000, 2000, device=dev)
            _ = x @ y
            torch.cuda.synchronize()
            t0 = time.time()
            r = x @ y
            torch.cuda.synchronize()
            print("matmul elapsed s:", round(time.time() - t0, 3))
            _ = r.sum().item()
            return True
        return False
    except Exception as e:
        print("pytorch test error:", e)
        return False


def check_cudnn_compatibility() -> bool:
    """Check CuDNN version compatibility between PyTorch and JAX."""
    print_section("CUDNN COMPATIBILITY CHECK")
    try:
        import torch
        import subprocess
        import glob
        
        # Check PyTorch CuDNN version
        pytorch_cudnn = torch.backends.cudnn.version()
        print(f"PyTorch CuDNN version: {pytorch_cudnn}")
        
        # Check installed nvidia-cudnn-cu12 package version
        try:
            result = subprocess.run(['uv', 'pip', 'list'], capture_output=True, text=True)
            if result.returncode == 0:
                lines = result.stdout.split('\n')
                for line in lines:
                    if 'nvidia-cudnn-cu12' in line:
                        print(f"Installed CuDNN package: {line.strip()}")
                        break
        except Exception as e:
            print(f"Could not check CuDNN package version: {e}")
        
        # Check CuDNN library files
        cudnn_paths = [
            "/app/.venv/lib/python3.10/site-packages/nvidia/cudnn/lib",
            "/usr/local/cuda/lib64",
            "/usr/lib/x86_64-linux-gnu"
        ]
        
        print("\nCuDNN library search:")
        for path in cudnn_paths:
            if os.path.exists(path):
                cudnn_libs = glob.glob(f"{path}/libcudnn*")
                if cudnn_libs:
                    print(f"  {path}: {len(cudnn_libs)} CuDNN libraries found")
                    for lib in cudnn_libs[:3]:  # Show first 3
                        print(f"    - {os.path.basename(lib)}")
                else:
                    print(f"  {path}: No CuDNN libraries found")
            else:
                print(f"  {path}: Path does not exist")
        
        # Check LD_LIBRARY_PATH
        ld_path = os.environ.get('LD_LIBRARY_PATH', '')
        print(f"\nLD_LIBRARY_PATH: {ld_path}")
        
        # Version compatibility check
        if pytorch_cudnn < 9000:  # Assuming version format like 9100 for 9.1.0
            print("WARNING: PyTorch CuDNN version may be too old for JAX")
            return False
        
        print("CuDNN compatibility check passed")
        return True
        
    except Exception as e:
        print(f"CuDNN compatibility check failed: {e}")
        return False


def test_jax_initialization() -> bool:
    print_section("JAX INITIALIZATION TEST")
    try:
        os.environ['JAX_TRACEBACK_FILTERING'] = 'off'
        import jax
        import jaxlib
        from jaxlib import xla_client
        print("jax:", jax.__version__, "jaxlib:", jaxlib.__version__)
        
        # Check for CuDNN version mismatch errors
        try:
            opts = xla_client.generate_pjrt_gpu_plugin_options()
            print("gpu plugin options ok; memory_fraction:", opts.get('memory_fraction', 'not set'))
        except Exception as e:
            print("gpu plugin options error:", e)
            if "could not convert string to float" in str(e):
                print("hint: check XLA_PYTHON_CLIENT_MEM_FRACTION for inline comments")
            elif "CuDNN" in str(e) and "version" in str(e):
                print("hint: CuDNN version mismatch detected - check compatibility")
            return False
        return True
    except Exception as e:
        print("jax init error:", e)
        if "CuDNN" in str(e):
            print("hint: CuDNN-related error detected")
        return False


def test_jax() -> bool:
    print_section("JAX GPU TEST")
    try:
        os.environ['JAX_TRACEBACK_FILTERING'] = 'off'
        import jax, jax.numpy as jnp
        from jax.lib import xla_bridge
        print("backend:", xla_bridge.get_backend().platform)
        devices = jax.devices()
        print("devices:", devices)
        gpus = [d for d in devices if 'gpu' in str(d).lower() or getattr(d, 'platform', '') == 'gpu']
        if not gpus:
            print("no gpu devices detected by jax")
            return False
        # quick compute
        import time
        key = jax.random.PRNGKey(0)
        x = jax.random.normal(key, (2000, 2000))
        x = jax.device_put(x, gpus[0])
        t0 = time.time()
        s = jnp.sum(x @ x).block_until_ready()
        print("matmul elapsed s:", round(time.time() - t0, 3), "sum:", float(s))
        return True
    except Exception as e:
        print("jax test error:", e)
        return False


def main() -> int:
    import argparse
    p = argparse.ArgumentParser()
    p.add_argument('--quick', action='store_true')
    p.add_argument('--fix', action='store_true', help='Run with fix recommendations')
    args = p.parse_args()

    if args.quick:
        env_ok = validate_environment_variables()
        pt_ok = test_pytorch()
        return 0 if (env_ok and pt_ok) else 1

    env_ok = validate_environment_variables()
    check_environment()
    cudnn_ok = check_cudnn_compatibility()
    jax_init_ok = test_jax_initialization()
    jax_ok = test_jax()
    pt_ok = test_pytorch()

    print_section("SUMMARY")
    print("env vars:", "ok" if env_ok else "fail")
    print("cudnn compatibility:", "ok" if cudnn_ok else "fail")
    print("jax init:", "ok" if jax_init_ok else "fail")
    print("jax compute:", "ok" if jax_ok else "fail")
    print("pytorch:", "ok" if pt_ok else "fail")

    # Provide fix recommendations if requested
    if args.fix and not (env_ok and cudnn_ok and jax_init_ok and jax_ok and pt_ok):
        print_section("FIX RECOMMENDATIONS")
        if not cudnn_ok:
            print("1. CuDNN version mismatch detected:")
            print("   - Upgrade nvidia-cudnn-cu12 to version >= 9.8.0")
            print("   - Ensure LD_LIBRARY_PATH includes CuDNN library paths")
        if not jax_init_ok:
            print("2. JAX initialization failed:")
            print("   - Check CuDNN compatibility")
            print("   - Verify XLA environment variables (no inline comments)")
        if not jax_ok:
            print("3. JAX GPU computation failed:")
            print("   - Verify GPU is accessible")
            print("   - Check CUDA driver compatibility")

    return 0 if (env_ok and cudnn_ok and jax_init_ok and jax_ok and pt_ok) else 1


if __name__ == '__main__':
    sys.exit(main())


Overwriting ../../.devcontainer/validate_gpu.py


In [8]:
%%writefile ../../.devcontainer/tests/test_pytorch_gpu.py
#!/usr/bin/env python3
"""Small PyTorch GPU benchmark."""
import time


def test_pytorch(force_cpu: bool = False) -> None:
    import torch
    cuda_ok = torch.cuda.is_available() and not force_cpu
    if cuda_ok:
        name = torch.cuda.get_device_name(0)
        major, minor = torch.cuda.get_device_capability()
        print(f"device: {name} (sm_{major}{minor:02d})")
        device = torch.device("cuda:0")
    else:
        print("falling back to cpu")
        device = torch.device("cpu")

    size = (1000, 1000)
    a, b = (torch.randn(size, device=device) for _ in range(2))
    _ = a @ b
    t0 = time.time()
    _ = (a @ b).sum().item()
    if device.type == "cuda":
        torch.cuda.synchronize()
    print(f"matmul on {device} took {(time.time()-t0)*1000:.2f} ms")


if __name__ == "__main__":
    test_pytorch()


Overwriting ../../.devcontainer/tests/test_pytorch_gpu.py


In [9]:
%%writefile ../../.devcontainer/tests/test_cv.py
#!/usr/bin/env python3
"""
Computer Vision validation and testing for Basketball Detection setup.
Tests YOLO, Roboflow, OpenCV, and tracking libraries.
"""
import sys
import os
import warnings
import numpy as np
warnings.filterwarnings('ignore')


def print_section(title: str) -> None:
    """Print a formatted section header."""
    print("\n" + "=" * 60)
    print(f"  {title}")
    print("=" * 60)


def test_opencv() -> bool:
    """Test OpenCV installation and basic functionality."""
    print_section("OPENCV TEST")
    try:
        import cv2
        print(f"OpenCV version: {cv2.__version__}")
        print(f"Build info: {cv2.getBuildInformation().split('General configuration')[0][:100]}...")
        
        # Check available video backends
        backends = []
        for backend in [cv2.CAP_FFMPEG, cv2.CAP_GSTREAMER, cv2.CAP_V4L2]:
            try:
                cap = cv2.VideoCapture()
                if cap.open(0, backend):
                    backends.append(backend)
                    cap.release()
            except:
                pass
        
        print(f"Available video backends: {backends}")
        
        # Test basic image operations
        test_img = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
        gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        
        print(f"Image processing test: OK (edges shape: {edges.shape})")
        return True
        
    except Exception as e:
        print(f"OpenCV test failed: {e}")
        return False


def test_ultralytics() -> bool:
    """Test Ultralytics YOLO installation and model loading."""
    print_section("ULTRALYTICS YOLO TEST")
    try:
        from ultralytics import YOLO, __version__
        from ultralytics.utils.checks import check_requirements
        
        print(f"Ultralytics version: {__version__}")
        
        # Check if CUDA is available for YOLO
        import torch
        print(f"YOLO CUDA available: {torch.cuda.is_available()}")
        
        # Test loading a model (will download if not cached)
        model_path = "/app/weights/yolov8n.pt"
        if not os.path.exists(model_path):
            print("Downloading YOLOv8n model...")
            model = YOLO("yolov8n.pt")
        else:
            print(f"Loading cached model from {model_path}")
            model = YOLO(model_path)
        
        print(f"Model loaded: {model.model.__class__.__name__}")
        print(f"Model device: {model.device}")
        
        # Test inference on dummy image
        dummy_img = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
        results = model(dummy_img, verbose=False)
        
        print(f"Inference test: OK (processed {len(results)} image)")
        return True
        
    except Exception as e:
        print(f"Ultralytics test failed: {e}")
        return False


def test_roboflow() -> bool:
    """Test Roboflow installation and API connectivity."""
    print_section("ROBOFLOW TEST")
    try:
        import roboflow
        from roboflow import Roboflow
        
        print(f"Roboflow version: {roboflow.__version__}")
        
        # Check if API key is set
        api_key = os.environ.get('ROBOFLOW_API_KEY', '')
        if api_key:
            print("Roboflow API key: Set")
            
            # Test API connection (won't actually download without valid key)
            try:
                rf = Roboflow(api_key=api_key)
                print("Roboflow client initialized successfully")
                
                # Try to access basketball court detection project
                workspace = os.environ.get('ROBOFLOW_WORKSPACE', 'basketball-formations')
                project = os.environ.get('ROBOFLOW_PROJECT', 'basketball-court-detection-2-mlopt')
                version = os.environ.get('ROBOFLOW_VERSION', '1')
                
                print(f"Workspace: {workspace}")
                print(f"Project: {project}")
                print(f"Version: {version}")
                
                # Note: This will fail without valid API key, which is expected
                # project = rf.workspace(workspace).project(project)
                # dataset = project.version(version)
                # print(f"Dataset accessible: {dataset.name}")
                
            except Exception as e:
                print(f"Note: Full Roboflow test requires valid API key: {e}")
        else:
            print("Roboflow API key: Not set (set ROBOFLOW_API_KEY to enable)")
            print("Get your API key from: https://app.roboflow.com/settings/api")
        
        return True
        
    except Exception as e:
        print(f"Roboflow test failed: {e}")
        return False


def test_supervision() -> bool:
    """Test supervision library for tracking and annotation."""
    print_section("SUPERVISION TRACKING TEST")
    try:
        import supervision as sv
        
        print(f"Supervision version: {sv.__version__}")
        
        # Test basic detection utilities
        from supervision import Detections, BoxAnnotator
        
        # Create dummy detections
        xyxy = np.array([[100, 100, 200, 200], [300, 300, 400, 400]])
        confidence = np.array([0.9, 0.8])
        class_id = np.array([0, 1])
        
        detections = Detections(
            xyxy=xyxy,
            confidence=confidence,
            class_id=class_id
        )
        
        print(f"Created {len(detections)} dummy detections")
        
        # Test annotator
        annotator = BoxAnnotator()
        print("Box annotator initialized")
        
        # Test tracker availability
        try:
            from supervision import ByteTrack
            tracker = ByteTrack()
            print("ByteTrack tracker available")
        except ImportError:
            print("ByteTrack tracker not available (optional)")
        
        return True
        
    except Exception as e:
        print(f"Supervision test failed: {e}")
        return False


def test_video_processing() -> bool:
    """Test video processing capabilities."""
    print_section("VIDEO PROCESSING TEST")
    try:
        # Test ffmpeg-python
        import ffmpeg
        print("ffmpeg-python: Available")
        
        # Check ffmpeg binary
        import subprocess
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            ffmpeg_version = result.stdout.split('\n')[0]
            print(f"ffmpeg binary: {ffmpeg_version}")
        else:
            print("ffmpeg binary: Not found or error")
            
        # Test moviepy
        try:
            import moviepy
            print(f"MoviePy version: {moviepy.__version__}")
        except Exception as e:
            print(f"MoviePy: {e}")
            
        # Test yt-dlp
        try:
            import yt_dlp
            print(f"yt-dlp version: {yt_dlp.version.__version__}")
        except Exception as e:
            print(f"yt-dlp: {e}")
            
        return True
        
    except Exception as e:
        print(f"Video processing test failed: {e}")
        return False


def test_basketball_models() -> bool:
    """Test basketball-specific model configurations."""
    print_section("BASKETBALL MODELS CONFIGURATION")
    
    print("Basketball Detection Models:")
    print("1. Court Detection: roboflow/basketball-court-detection-2")
    print("2. Player Detection: YOLOv8 (person class)")
    print("3. Ball Detection: Custom YOLO model")
    print("4. Jersey Number OCR: Available via Roboflow")
    
    # Check model directories
    model_dirs = ['/app/models', '/app/weights', '/workspace/models']
    for dir_path in model_dirs:
        if os.path.exists(dir_path):
            files = os.listdir(dir_path)
            if files:
                print(f"\n{dir_path}: {len(files)} files")
                for f in files[:5]:  # Show first 5 files
                    print(f"  - {f}")
            else:
                print(f"\n{dir_path}: Empty")
        else:
            print(f"\n{dir_path}: Not created yet")
    
    return True


def main() -> int:
    """Run all computer vision tests."""
    import argparse
    parser = argparse.ArgumentParser(description="Test Computer Vision components")
    parser.add_argument('--quick', action='store_true', help='Run quick tests only')
    parser.add_argument('--verbose', action='store_true', help='Verbose output')
    args = parser.parse_args()
    
    if args.verbose:
        print_section("ENVIRONMENT VARIABLES")
        cv_vars = ['ROBOFLOW_API_KEY', 'YOLO_VERBOSE', 'OPENCV_LOG_LEVEL', 
                   'VIDEO_INPUT_DIR', 'VIDEO_OUTPUT_DIR', 'DISPLAY']
        for var in cv_vars:
            value = os.environ.get(var, 'Not set')
            if var == 'ROBOFLOW_API_KEY' and value != 'Not set':
                value = value[:10] + '...' if len(value) > 10 else value
            print(f"{var}: {value}")
    
    # Run tests
    opencv_ok = test_opencv()
    yolo_ok = test_ultralytics()
    
    if args.quick:
        print_section("QUICK TEST SUMMARY")
        print(f"OpenCV: {'✓' if opencv_ok else '✗'}")
        print(f"YOLO: {'✓' if yolo_ok else '✗'}")
        return 0 if (opencv_ok and yolo_ok) else 1
    
    roboflow_ok = test_roboflow()
    supervision_ok = test_supervision()
    video_ok = test_video_processing()
    basketball_ok = test_basketball_models()
    
    print_section("COMPUTER VISION TEST SUMMARY")
    results = {
        "OpenCV": opencv_ok,
        "YOLO/Ultralytics": yolo_ok,
        "Roboflow": roboflow_ok,
        "Supervision": supervision_ok,
        "Video Processing": video_ok,
        "Basketball Config": basketball_ok
    }
    
    for component, status in results.items():
        status_symbol = "✓" if status else "✗"
        print(f"{component}: {status_symbol}")
    
    all_ok = all(results.values())
    
    if not all_ok:
        print("\n⚠️  Some components failed. Check the logs above for details.")
        print("Common fixes:")
        print("  - Set ROBOFLOW_API_KEY for Roboflow integration")
        print("  - Ensure ffmpeg is installed for video processing")
        print("  - Check GPU drivers for YOLO acceleration")
    else:
        print("\n✅ All computer vision components are working correctly!")
    
    return 0 if all_ok else 1


if __name__ == '__main__':
    sys.exit(main())


Overwriting ../../.devcontainer/tests/test_cv.py


In [10]:
%%writefile ../../.devcontainer/tests/test_yolo_bas
#!/usr/bin/env python3
"""
YOLO Basketball Detection Test
Tests YOLO functionality on basketball images to ensure the package works correctly.
"""
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch
import sys
from pathlib import Path
import requests
from urllib.parse import urlparse
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class YOLOBasketballTester:
    """Test YOLO functionality with basketball-specific scenarios."""
    
    def __init__(self, model_name="yolov8n.pt", force_cpu=False):
        """Initialize the YOLO tester."""
        self.model_name = model_name
        self.force_cpu = force_cpu
        self.model = None
        self.device = "cpu" if force_cpu else ("cuda" if torch.cuda.is_available() else "cpu")
        
        # Create directories for test data
        self.data_dir = Path("/workspace/data")
        self.images_dir = self.data_dir / "images"
        self.output_dir = self.data_dir / "output"
        
        for dir_path in [self.data_dir, self.images_dir, self.output_dir]:
            dir_path.mkdir(exist_ok=True, parents=True)
        
        self.test_image_path = self.images_dir / "basketball_test.jpg"
        
    def setup_test_environment(self):
        """Set up the test environment and download test image if needed."""
        print("=" * 60)
        print("YOLO BASKETBALL DETECTION TEST")
        print("=" * 60)
        
        # Check environment
        print(f"Python version: {sys.version}")
        print(f"OpenCV version: {cv2.__version__}")
        print(f"PyTorch version: {torch.__version__}")
        print(f"CUDA available: {torch.cuda.is_available()}")
        print(f"Device selected: {self.device}")
        
        # Get or create test image
        if not self.test_image_path.exists():
            print(f"\nTest image not found at {self.test_image_path}")
            self._create_or_download_test_image()
        else:
            print(f"Using existing test image: {self.test_image_path}")
    
    def _create_or_download_test_image(self):
        """Create or download a basketball test image."""
        print("Creating synthetic basketball test image...")
        
        # Create a synthetic basketball court scene
        image = self._create_synthetic_basketball_scene()
        
        # Save the image
        cv2.imwrite(str(self.test_image_path), image)
        print(f"Created synthetic test image: {self.test_image_path}")
        
        return str(self.test_image_path)
    
    def _create_synthetic_basketball_scene(self):
        """Create a synthetic basketball scene for testing."""
        # Create a 1280x720 image (HD resolution)
        width, height = 1280, 720
        image = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Basketball court background (brown/tan)
        court_color = (139, 115, 85)  # Brown color in BGR
        image[:] = court_color
        
        # Draw court lines (white)
        line_color = (255, 255, 255)
        line_thickness = 3
        
        # Center line
        cv2.line(image, (width//2, 0), (width//2, height), line_color, line_thickness)
        
        # Center circle
        cv2.circle(image, (width//2, height//2), 100, line_color, line_thickness)
        
        # Free throw lines
        ft_line_y = height // 4
        cv2.line(image, (0, ft_line_y), (width, ft_line_y), line_color, line_thickness)
        cv2.line(image, (0, height - ft_line_y), (width, height - ft_line_y), line_color, line_thickness)
        
        # Add some "players" (rectangles representing people)
        player_color = (255, 0, 0)  # Blue jerseys
        player_positions = [
            (200, 300, 60, 120),  # x, y, width, height
            (400, 400, 60, 120),
            (800, 350, 60, 120),
            (1000, 250, 60, 120),
            (600, 500, 60, 120),
        ]
        
        for x, y, w, h in player_positions:
            cv2.rectangle(image, (x, y), (x + w, y + h), player_color, -1)
            # Add head (circle)
            cv2.circle(image, (x + w//2, y - 20), 15, (255, 200, 150), -1)
        
        # Add a "basketball" (orange circle)
        ball_center = (640, 360)  # Center of image
        ball_radius = 15
        ball_color = (0, 165, 255)  # Orange in BGR
        cv2.circle(image, ball_center, ball_radius, ball_color, -1)
        
        # Add some lines to make it look more like a basketball
        cv2.line(image, (ball_center[0] - ball_radius, ball_center[1]), 
                (ball_center[0] + ball_radius, ball_center[1]), (0, 0, 0), 2)
        cv2.line(image, (ball_center[0], ball_center[1] - ball_radius), 
                (ball_center[0], ball_center[1] + ball_radius), (0, 0, 0), 2)
        
        return image
    
    def load_yolo_model(self):
        """Load the YOLO model."""
        print(f"\nLoading YOLO model: {self.model_name}")
        
        try:
            # Load model
            self.model = YOLO(self.model_name)
            
            # Move to appropriate device
            if self.device == "cuda":
                self.model.to('cuda')
            
            print(f"Model loaded successfully on {self.device}")
            print(f"Model classes: {len(self.model.names)} total")
            
            # Show relevant classes for basketball

%%writefile ../../.devcontainer/validate_gpu.py
#!/usr/bin/env python3
"""
YOLO Basketball Detection Test
Tests YOLO functionality on basketball images to ensure the package works correctly.
"""
            for class_id, class_name in relevant_classes.items():
                if class_id in self.model.names:
                    print(f"  {class_id}: {self.model.names[class_id]}")
            
            return True
            
        except Exception as e:
            print(f"Error loading YOLO model: {e}")
            return False
    
    def test_yolo_detection(self):
        """Test YOLO detection on the basketball image."""
        print(f"\nTesting YOLO detection on {self.test_image_path}")
        
        # Load image
        image = cv2.imread(str(self.test_image_path))
        if image is None:
            print(f"Error: Could not load image from {self.test_image_path}")
            return False
        
        print(f"Image loaded: {image.shape}")
        
        try:
            # Run detection
            print("Running YOLO detection...")
            results = self.model(
                image,
                conf=0.25,  # Lower confidence for testing
                iou=0.45,
                verbose=False
            )
            
            # Process results
            detections = results[0]
            
            print(f"Detection completed!")
            print(f"Number of detections: {len(detections.boxes) if detections.boxes is not None else 0}")
            
            # Analyze detections
            detection_summary = self._analyze_detections(detections)
            
            # Create annotated image
            annotated_image = self._create_annotated_image(image, detections)
            
            # Save results
            output_path = self.output_dir / "basketball_test_result.jpg"
            cv2.imwrite(str(output_path), annotated_image)
            print(f"Annotated result saved to: {output_path}")
            
            return True, detection_summary, str(output_path)
            
        except Exception as e:
            print(f"Error during YOLO detection: {e}")
            return False, None, None
    
    def _analyze_detections(self, detections):
        """Analyze detection results."""
        summary = {
            'total_detections': 0,
            'persons': 0,
            'sports_balls': 0,
            'other_objects': 0,
            'confidence_scores': [],
            'class_names': []
        }
        
        if detections.boxes is None:
            print("No detections found.")
            return summary
        
        boxes = detections.boxes
        summary['total_detections'] = len(boxes)
        
        for i in range(len(boxes)):
            class_id = int(boxes.cls[i])
            confidence = float(boxes.conf[i])
            class_name = self.model.names[class_id]
            
            summary['confidence_scores'].append(confidence)
            summary['class_names'].append(class_name)
            
            if class_id == 0:  # person
                summary['persons'] += 1
            elif class_id == 32:  # sports ball
                summary['sports_balls'] += 1
            else:
                summary['other_objects'] += 1
            
            print(f"  Detection {i+1}: {class_name} (confidence: {confidence:.3f})")
        
        return summary
    
    def _create_annotated_image(self, image, detections):
        """Create annotated image with detection results."""
        annotated = image.copy()
        
        if detections.boxes is None:
            return annotated
        
        boxes = detections.boxes
        
        for i in range(len(boxes)):
            # Get box coordinates
            x1, y1, x2, y2 = boxes.xyxy[i].int().tolist()
            class_id = int(boxes.cls[i])
            confidence = float(boxes.conf[i])
            class_name = self.model.names[class_id]
            
            # Choose color based on class
            if class_id == 0:  # person
                color = (255, 0, 0)  # Blue
            elif class_id == 32:  # sports ball
                color = (0, 255, 255)  # Yellow
            else:
                color = (0, 255, 0)  # Green
            
            # Draw bounding box
            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
            
            # Draw label
            label = f"{class_name}: {confidence:.2f}"
            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
            cv2.rectangle(annotated, (x1, y1 - label_size[1] - 10), 
                         (x1 + label_size[0], y1), color, -1)
            cv2.putText(annotated, label, (x1, y1 - 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return annotated
    
    def display_results(self, image_path, annotated_path):
        """Display the original and annotated images."""
        try:
            # Load images
            original = cv2.imread(image_path)
            annotated = cv2.imread(annotated_path)
            
            # Convert BGR to RGB for matplotlib
            original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
            annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
            
            # Create subplot
            fig, axes = plt.subplots(1, 2, figsize=(15, 7))
            
            axes[0].imshow(original_rgb)
            axes[0].set_title("Original Basketball Test Image")
            axes[0].axis('off')
            
            axes[1].imshow(annotated_rgb)
            axes[1].set_title("YOLO Detection Results")
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"Error displaying results: {e}")
    
    def run_full_test(self):
        """Run the complete YOLO basketball test."""
        print("Starting YOLO Basketball Detection Test...")
        
        # Step 1: Setup environment
        self.setup_test_environment()
        
        # Step 2: Load YOLO model
        if not self.load_yolo_model():
            print("Failed to load YOLO model. Test aborted.")
            return False
        
        # Step 3: Test detection
        success, summary, output_path = self.test_yolo_detection()
        
        if not success:
            print("YOLO detection test failed.")
            return False
        
        # Step 4: Display results
        print("\n" + "=" * 60)
        print("TEST RESULTS SUMMARY")
        print("=" * 60)
        print(f"Total detections: {summary['total_detections']}")
        print(f"Persons detected: {summary['persons']}")
        print(f"Sports balls detected: {summary['sports_balls']}")
        print(f"Other objects detected: {summary['other_objects']}")
        
        if summary['confidence_scores']:
            avg_confidence = np.mean(summary['confidence_scores'])
            print(f"Average confidence: {avg_confidence:.3f}")
        
        print(f"Results saved to: {output_path}")
        
        # Step 5: Visual display (if in Jupyter/interactive environment)
        try:
            self.display_results(str(self.test_image_path), output_path)
        except:
            print("Note: Visual display not available (likely running in non-interactive mode)")
        
        print("\n✅ YOLO Basketball Detection Test COMPLETED SUCCESSFULLY!")
        return True


def main():
    """Main function to run the test."""
    import argparse
    
    parser = argparse.ArgumentParser(description="Test YOLO on basketball images")
    parser.add_argument("--model", default="yolov8n.pt", help="YOLO model to use")
    parser.add_argument("--cpu", action="store_true", help="Force CPU usage")
    parser.add_argument("--image", help="Path to custom test image")
    
    args = parser.parse_args()
    
    # Create tester
    tester = YOLOBasketballTester(model_name=args.model, force_cpu=args.cpu)
    
    # Use custom image if provided
    if args.image and os.path.exists(args.image):
        tester.test_image_path = Path(args.image)
        print(f"Using custom test image: {args.image}")
    
    # Run test
    success = tester.run_full_test()
    
    return 0 if success else 1


if __name__ == "__main__":
    exit(main())

Writing ../../.devcontainer/tests/test_yolo_bas


In [11]:
%%writefile ../../.devcontainer/tests/test_pytorch.py
print("PyTorch quick check")
try:
    import torch
    print("version:", torch.__version__)
    print("cuda:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("devices:", torch.cuda.device_count())
        for i in range(torch.cuda.device_count()):
            print(i, torch.cuda.get_device_name(i))
        x = torch.ones(100, 100, device='cuda:0')
        print("sum:", float(torch.sum(x)))
except Exception as e:
    print("error:", e)


Overwriting ../../.devcontainer/tests/test_pytorch.py


In [12]:
%%writefile ../../.devcontainer/tests/test_uv.py
# Test other critical packages
print("\n📦 Testing other critical packages...")

packages_to_test = [
    'numpy', 'pandas', 'matplotlib', 'scipy', 'sklearn', 
    'jupyterlab', 'seaborn', 'tqdm'
]

for package in packages_to_test:
    try:
        if package == 'sklearn':
            import sklearn
            version = sklearn.__version__
        else:
            module = __import__(package)
            version = getattr(module, '__version__', 'unknown')
        print(f"   ✅ {package}: {version}")
    except ImportError:
        print(f"   ❌ {package}: Not installed")
    except Exception as e:
        print(f"   ⚠️  {package}: Error - {e}")


Overwriting ../../.devcontainer/tests/test_uv.py


In [13]:
%%writefile ../../.devcontainer/tests/test_summary.py
#!/usr/bin/env python3
"""Aggregated checks for the devcontainer layout and GPU readiness."""
import os
import sys
import time
import subprocess


def section(t):
    print("\n" + "=" * 60)
    print(t)
    print("=" * 60)


def test_structure() -> bool:
    section("STRUCTURE")
    expected = [
        '/workspace/docker-compose.yml',
        '/workspace/pyproject.toml',
        '/workspace/.devcontainer/devcontainer.json',
        '/workspace/.devcontainer/Dockerfile',
        '/workspace/.devcontainer/.env.template',
        '/workspace/.devcontainer/.dockerignore',
        '/app/validate_gpu.py',
        '/app/tests/'
    ]
    ok = True
    for p in expected:
        if os.path.exists(p):
            print("ok:", p)
        else:
            print("missing:", p)
            ok = False
    return ok


def test_uv() -> bool:
    section("UV")
    try:
        r = subprocess.run(['uv', '--version'], capture_output=True, text=True)
        print(r.stdout.strip() or r.stderr.strip())
        return r.returncode == 0
    except FileNotFoundError:
        print('uv not in PATH')
        return False


def test_pytorch() -> bool:
    section("PYTORCH")
    try:
        import torch
        print("version:", torch.__version__)
        print("cuda:", torch.cuda.is_available())
        if torch.cuda.is_available():
            d = torch.device('cuda:0')
            x = torch.ones(512, 512, device=d)
            y = torch.sum(x)
            print("sum:", y.item())
            return True
        return False
    except Exception as e:
        print("error:", e)
        return False


def test_jax() -> bool:
    section("JAX")
    try:
        import jax, jax.numpy as jnp

        # Show all devices for visibility
        devs = jax.devices()
        print("devices:", devs)

        # Prefer the supported filtered query
        gpus = jax.devices("gpu")

        # Fallback for older/newer renderings (e.g., "CudaDevice(id=0)")
        if not gpus:
            gpus = [
                d for d in devs
                if getattr(d, "platform", "").lower() in {"gpu", "cuda"} or "cuda" in str(d).lower()
            ]

        if not gpus:
            print("no gpu devices detected by jax")
            return False

        # Tiny compute on the first GPU to ensure execution
        x = jnp.ones((512, 512), dtype=jnp.float32)
        x = jax.device_put(x, gpus[0])
        s = jnp.sum(x).block_until_ready()
        print("sum:", float(s))
        return True
    except Exception as e:
        print("error:", e)
        return False



def main() -> int:
    s_ok = test_structure()
    uv_ok = test_uv()
    pt_ok = test_pytorch()
    j_ok = test_jax()

    section("SUMMARY")
    print("structure:", s_ok, "uv:", uv_ok, "pytorch:", pt_ok, "jax:", j_ok)
    return 0 if all([s_ok, uv_ok, pt_ok, j_ok]) else 1


if __name__ == '__main__':
    sys.exit(main())

Overwriting ../../.devcontainer/tests/test_summary.py
